# chatgpt_transformer_shakespeare.ipynb
# WESmith 06/22/23
## experiment with chatgpt source code in writing a transformer to generate text
## status: this not finished: will need to add a batch mode and run on a GPU

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import string
import random
import math
import numpy as np

In [ ]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Define the hyperparameters
input_size = len(string.ascii_letters) + len(string.punctuation) + 1
hidden_size = 128
output_size = input_size
num_layers = 4
num_heads = 8
dropout = 0.1
learning_rate = 0.01
num_epochs = 1000

In [ ]:
# get Shakespeare text
with open('input.txt', 'r', encoding='utf-8') as f:
    shakespeare_text = f.read()

In [ ]:
all_characters = sorted(list(set(shakespeare_text)))
vocab_size = len(all_characters)
vocab_size, len(shakespeare_text)

In [ ]:
''.join([c for c in all_characters])

In [ ]:
shakespeare_text[:100]

In [ ]:
# Prepare the training data (Shakespeare text)
#shakespeare_text = "YOUR_SHAKESPEARE_TEXT_HERE"
#all_characters = string.ascii_letters + string.punctuation + ' '
char_to_idx = {char: i for i, char in enumerate(all_characters)}
idx_to_char = {i: char for i, char in enumerate(all_characters)}

# Convert the text to a tensor
full_text_tensor = torch.tensor([[char_to_idx[char] for char in shakespeare_text]], 
                           dtype=torch.long).to(device)

In [ ]:
full_text_tensor.shape

In [1]:
# batches not used in initial chatgpt result, limit number of text chars and run as one batch
limit = 1000
text_tensor = full_text_tensor[0, :limit]
text_tensor = text_tensor.unsqueeze(0)
text_tensor.shape, full_text_tensor.shape

NameError: name 'full_text_tensor' is not defined

In [ ]:
# Function to generate a sample of text from the generator model
def generate_sample(start_char='A', length=100):
    with torch.no_grad():
        input_chars = torch.tensor([[char_to_idx[start_char]]], dtype=torch.long).to(device)

        generated_text = start_char

        for _ in range(length):
            output = generator(input_chars)
            _, top_char_idx = output[:, -1, :].topk(1)
            generated_char = idx_to_char[top_char_idx.item()]

            generated_text += generated_char

            input_chars = torch.cat((input_chars, top_char_idx.unsqueeze(0)), dim=1)

        return generated_text

In [ ]:
# Define the Generator model using Transformer architecture
class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, num_heads, dropout):
        super(Generator, self).__init__()
        self.hidden_size = hidden_size

        self.embedding   = nn.Embedding(input_size, hidden_size)
        self.transformer = nn.Transformer(d_model=hidden_size, nhead=num_heads, 
                                          num_encoder_layers=num_layers, 
                                          num_decoder_layers=num_layers, 
                                          dim_feedforward=hidden_size, dropout=dropout)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, input_chars):
        input_embedded = self.embedding(input_chars)
        input_embedded = input_embedded.permute(1, 0, 2)  # Reshape for Transformer input

        output = self.transformer(input_embedded, input_embedded)
        output = self.softmax(self.out(output))

        return output

In [ ]:
# Instantiate the generator model
generator = Generator(input_size, hidden_size, output_size, 
                      num_layers, num_heads, dropout).to(device)

In [ ]:
# WS added to see model size
num = sum(p.numel() for p in generator.parameters())
print(f'The model has {num:.5g} parameters.')

In [ ]:
# Define the loss function and optimizer
criterion = nn.NLLLoss()
optimizer = optim.SGD(generator.parameters(), lr=learning_rate)

In [ ]:
text_tensor.shape

In [ ]:
# Train the generator model
for epoch in range(num_epochs):
    loss = 0
    generator.zero_grad()

    output = generator(text_tensor[:, :-1])
    target = text_tensor[:, 1:]
    loss = criterion(output.view(-1, output_size), target.view(-1))

    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch: {epoch+1}, Loss: {loss.item()}')

In [ ]:
# Generate a sample of Shakespeare-like text
sample_text = generate_sample(start_char='A', length=200)
print(sample_text)